In [7]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [8]:
!wget -q !wget -q https://dlcdn.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz
!pip install -q findspark

In [9]:
!pip install pyspark

In [10]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

In [11]:
import findspark
findspark.init()

In [12]:
# Iniciar uma sessão local e importar dados
from pyspark.sql import SparkSession
sc = SparkSession.builder.master('local[*]').getOrCreate()

In [13]:
# carregar dados do meu computador
dados_spark = sc.read.csv("/content/gasto_cliente.csv", header=True)

# Verificando o tipo de objeto criado
type(dados_spark)

pyspark.sql.dataframe.DataFrame

In [14]:
# Verificando o dataset
dados_spark.show(5)

+----------+----------+------+-----+--------+----------------+--------------------------+------------+-------------------+-------------------+-------------------+-----+
|ID_Usuario|ID_Produto|Genero|Idade|Ocupacao|Cidade_Categoria|Permanencia_na_Cidade_Anos|Estado_Civil|Categoria_Produto_1|Categoria_Produto_2|Categoria_Produto_3|Valor|
+----------+----------+------+-----+--------+----------------+--------------------------+------------+-------------------+-------------------+-------------------+-----+
|   1000001| P00069042|     F| 0-17|      10|               A|                         2|           0|                  3|               null|               null| 8370|
|   1000001| P00248942|     F| 0-17|      10|               A|                         2|           0|                  1|                  6|                 14|15200|
|   1000001| P00087842|     F| 0-17|      10|               A|                         2|           0|                 12|               null|             

In [15]:
# Verificando o schema() deste spark dataframe
dados_spark.printSchema()

root
 |-- ID_Usuario: string (nullable = true)
 |-- ID_Produto: string (nullable = true)
 |-- Genero: string (nullable = true)
 |-- Idade: string (nullable = true)
 |-- Ocupacao: string (nullable = true)
 |-- Cidade_Categoria: string (nullable = true)
 |-- Permanencia_na_Cidade_Anos: string (nullable = true)
 |-- Estado_Civil: string (nullable = true)
 |-- Categoria_Produto_1: string (nullable = true)
 |-- Categoria_Produto_2: string (nullable = true)
 |-- Categoria_Produto_3: string (nullable = true)
 |-- Valor: string (nullable = true)



In [16]:
#Retornar o número de linhas
dados_spark.count()

550068

In [17]:
#Selecionar colunas
dados_spark.select("ID_Usuario","Genero","Idade","Ocupacao").show(5)

+----------+------+-----+--------+
|ID_Usuario|Genero|Idade|Ocupacao|
+----------+------+-----+--------+
|   1000001|     F| 0-17|      10|
|   1000001|     F| 0-17|      10|
|   1000001|     F| 0-17|      10|
|   1000001|     F| 0-17|      10|
|   1000002|     M|  55+|      16|
+----------+------+-----+--------+
only showing top 5 rows



In [18]:
#Principais estatísticas
dados_spark.describe().show(5)

+-------+------------------+----------+------+------+-----------------+----------------+--------------------------+-------------------+-------------------+-------------------+-------------------+-----------------+
|summary|        ID_Usuario|ID_Produto|Genero| Idade|         Ocupacao|Cidade_Categoria|Permanencia_na_Cidade_Anos|       Estado_Civil|Categoria_Produto_1|Categoria_Produto_2|Categoria_Produto_3|            Valor|
+-------+------------------+----------+------+------+-----------------+----------------+--------------------------+-------------------+-------------------+-------------------+-------------------+-----------------+
|  count|            550068|    550068|550068|550068|           550068|          550068|                    550068|             550068|             550068|             376430|             166821|           550068|
|   mean|1003028.8424013031|      null|  null|  null|8.076706879876669|            null|         1.468494139793958|0.40965298835780306|  5.40427

In [19]:
#Ocorrências em uma variável
dados_spark.select("Cidade_Categoria").distinct().show()

+----------------+
|Cidade_Categoria|
+----------------+
|               B|
|               C|
|               A|
+----------------+



In [20]:
#Ocorrências em uma variável
dados_spark.select("Estado_Civil").distinct().show()

+------------+
|Estado_Civil|
+------------+
|           0|
|           1|
+------------+



In [21]:
#importar sql: groupBy
from pyspark.sql import functions as F

dados_spark.groupBy("Cidade_Categoria").agg(F.sum("Valor")).show()

+----------------+-------------+
|Cidade_Categoria|   sum(Valor)|
+----------------+-------------+
|               B|2.115533605E9|
|               C|1.663807476E9|
|               A|1.316471661E9|
+----------------+-------------+



In [22]:
item_popular = dados_spark.groupBy('ID_Produto').agg(F.count('ID_Produto')).orderBy(F.count('ID_Produto').desc()).show(10)

+----------+-----------------+
|ID_Produto|count(ID_Produto)|
+----------+-----------------+
| P00265242|             1880|
| P00025442|             1615|
| P00110742|             1612|
| P00112142|             1562|
| P00057642|             1470|
| P00184942|             1440|
| P00046742|             1438|
| P00058042|             1422|
| P00059442|             1406|
| P00145042|             1406|
+----------+-----------------+
only showing top 10 rows



In [23]:
idade_popular = dados_spark.groupBy('Idade').agg(F.sum('Valor')).orderBy(F.sum('Valor').desc()).show()

+-----+-------------+
|Idade|   sum(Valor)|
+-----+-------------+
|26-35|2.031770578E9|
|36-45|1.026569884E9|
|18-25| 9.13848675E8|
|46-50| 4.20843403E8|
|51-55| 3.67099644E8|
|  55+| 2.00767375E8|
| 0-17| 1.34913183E8|
+-----+-------------+



In [24]:
gasto_cliente = dados_spark.groupBy('ID_Usuario').agg(F.count('ID_Usuario'),F.sum('Valor')).orderBy(F.sum('Valor').desc()).show(10)

+----------+-----------------+-----------+
|ID_Usuario|count(ID_Usuario)| sum(Valor)|
+----------+-----------------+-----------+
|   1004277|              979|1.0536909E7|
|   1001680|             1026|  8699596.0|
|   1002909|              718|  7577756.0|
|   1001941|              898|  6817493.0|
|   1000424|              694|  6573609.0|
|   1004448|              575|  6566245.0|
|   1005831|              727|  6512433.0|
|   1001015|              740|  6511314.0|
|   1003391|              698|  6477160.0|
|   1001181|              862|  6387961.0|
+----------+-----------------+-----------+
only showing top 10 rows



In [25]:
#Dados faltantes
dados_spark.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in dados_spark.columns]).show()

+----------+----------+------+-----+--------+----------------+--------------------------+------------+-------------------+-------------------+-------------------+-----+
|ID_Usuario|ID_Produto|Genero|Idade|Ocupacao|Cidade_Categoria|Permanencia_na_Cidade_Anos|Estado_Civil|Categoria_Produto_1|Categoria_Produto_2|Categoria_Produto_3|Valor|
+----------+----------+------+-----+--------+----------------+--------------------------+------------+-------------------+-------------------+-------------------+-----+
|         0|         0|     0|    0|       0|               0|                         0|           0|                  0|             173638|             383247|    0|
+----------+----------+------+-----+--------+----------------+--------------------------+------------+-------------------+-------------------+-------------------+-----+



In [26]:
#substituir valores ausentes
dados_spark_sem_na = dados_spark.fillna({'Categoria_Produto_2':'Sem Resposta', 'Categoria_Produto_3':'Sem Resposta'})

In [27]:
dados_spark_sem_na.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in dados_spark_sem_na.columns]).show()

+----------+----------+------+-----+--------+----------------+--------------------------+------------+-------------------+-------------------+-------------------+-----+
|ID_Usuario|ID_Produto|Genero|Idade|Ocupacao|Cidade_Categoria|Permanencia_na_Cidade_Anos|Estado_Civil|Categoria_Produto_1|Categoria_Produto_2|Categoria_Produto_3|Valor|
+----------+----------+------+-----+--------+----------------+--------------------------+------------+-------------------+-------------------+-------------------+-----+
|         0|         0|     0|    0|       0|               0|                         0|           0|                  0|                  0|                  0|    0|
+----------+----------+------+-----+--------+----------------+--------------------------+------------+-------------------+-------------------+-------------------+-----+



In [28]:
# Verificando o novo dataset
dados_spark_sem_na.show(5)

+----------+----------+------+-----+--------+----------------+--------------------------+------------+-------------------+-------------------+-------------------+-----+
|ID_Usuario|ID_Produto|Genero|Idade|Ocupacao|Cidade_Categoria|Permanencia_na_Cidade_Anos|Estado_Civil|Categoria_Produto_1|Categoria_Produto_2|Categoria_Produto_3|Valor|
+----------+----------+------+-----+--------+----------------+--------------------------+------------+-------------------+-------------------+-------------------+-----+
|   1000001| P00069042|     F| 0-17|      10|               A|                         2|           0|                  3|       Sem Resposta|       Sem Resposta| 8370|
|   1000001| P00248942|     F| 0-17|      10|               A|                         2|           0|                  1|                  6|                 14|15200|
|   1000001| P00087842|     F| 0-17|      10|               A|                         2|           0|                 12|       Sem Resposta|       Sem Re

In [29]:
#Excluir colunas da base de dados
dados_spark = dados_spark.drop('Categoria_Produto_2','Categoria_Produto_3')
dados_spark.show(5)

+----------+----------+------+-----+--------+----------------+--------------------------+------------+-------------------+-----+
|ID_Usuario|ID_Produto|Genero|Idade|Ocupacao|Cidade_Categoria|Permanencia_na_Cidade_Anos|Estado_Civil|Categoria_Produto_1|Valor|
+----------+----------+------+-----+--------+----------------+--------------------------+------------+-------------------+-----+
|   1000001| P00069042|     F| 0-17|      10|               A|                         2|           0|                  3| 8370|
|   1000001| P00248942|     F| 0-17|      10|               A|                         2|           0|                  1|15200|
|   1000001| P00087842|     F| 0-17|      10|               A|                         2|           0|                 12| 1422|
|   1000001| P00085442|     F| 0-17|      10|               A|                         2|           0|                 12| 1057|
|   1000002| P00285442|     M|  55+|      16|               C|                        4+|        

In [32]:
# Salvar resultado
dados_spark_sem_na.write.csv("/content/drive/MyDrive/data")

In [33]:
# Spark para Pandas
dados_spark_sem_na_pd = dados_spark_sem_na.toPandas()

# Salvar resultado
dados_spark_sem_na_pd.to_csv("/content/drive/MyDrive/data.csv")

In [34]:
import matplotlib.pyplot as plt

In [ ]:
valor = dados_spark_sem_na_pd['Valor']
plt.hist(valor)
plt.show()